In [46]:
import numpy as np
import itertools

In [47]:
from konlpy.tag import Kkma, Komoran, Hannanum, Okt

In [49]:
from konlpy.utils import pprint

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
import pandas as pd

In [53]:
data = pd.read_csv("./fivecart_2020.csv", encoding='cp949')
data.head(5)

,report_writer,date_writer,title,book_writer,book_company,book_date,report_text
0,박유담,2022-05-02,군주론,마키아벨리,현대지성,2021-07-26,마키아벨리의 군주론을 읽고21세기를 살아가고 있는 우리에게 리더의 역할은 더욱...
1,백석,2022-04-17,빛 속으로 (한국 문학사에서 지워진 이름. 평생을 방랑자로 산 작가 김사량의 작품집),김사량,녹색광선,2021-08-15,<비뚤어진 우월감> 우리는 때때로 자신의 치부로 울타리를 두르고 그 안에 갇...
2,maobi,2022-04-16,작별하지 않는다 (한강 장편소설),한강,문학동네,2021-09-09,다시 한강 작가님의 책을 읽을 때까지는 고민이 있었다. '소년이 온다'를 읽...
3,킹고선비,2022-04-10,나의 한국현대사 1959-2020,유시민,돌베개,2021-01-20,59년생 ‘68세대’의 회고록을 읽은 97년생 ‘MZ세대’의 감상문 몇 ...
4,nama,2022-04-08,그냥 하지 말라 (당신의 모든 것이 메시지다),송길영,북스톤,2021-10-05,"저자는 과거부터 현재까지 변화해온, 또 앞으로 변화할 한국사회를 데이터를 통해..."


In [54]:
doc = data['report_text'][4]
doc

'   저자는 과거부터 현재까지 변화해온, 또 앞으로 변화할 한국사회를 데이터를 통해 들여다보고 이야기한다. 비슷한 결의 책을 여러 권 읽어봤지만 단순히 현상만 나열하는 게 아니라 저자의 통찰이 깊게 느껴진 책은 정말 오랜만이었다. 그러면서도 어렵지 않게 쓴다는 것이 대단하다고 생각했다. 세상에 똑똑한 사람은 정말 많지만 어려운 이야기를 이해하기 쉽게 전달하는 능력을 가진 사람은 그리 많지 않은 것 같다. 그래서 더 반갑고 좋았다.  이 책은 변화하는 사회를 살아가기 위해 사회구성원 개개인이 취해야 할 자세를 제시한다. 젊은 세대가 읽어도 좋지만 특히 관리자층이 이 책을 읽어봤으면 하는 바람이다. 중간관리자들의 무능과 고집을 날카롭게 지적하는 부분이 많았다. 나는 아직 사회생활을 제대로 시작해보지 않았지만 가족, 친구들과 대화하다보면 꼭 일도 못하면서 남에게 책임만 전가하는 상사 얘기가 나온다. 나는 이런 어른이 되지는 말아야지. 스스로를 경계하기 위해서도 한 번쯤 읽어보면 좋을 책이다. 중간중간 뜨끔하는 문장이 있을 수도 있다.[당신의 모든 것이 메시지다]책의 부제이기도 한 이 챕터가 가장 인상적이었다. 요즘 들어 기록의 중요성을 점점 체감하고 있다. 개인적으로 기억하기 위함도 있지만 어딘가에 내가 한 활동들과 내 생각을 보여줘야할 때 특히 더 절실히 느낀다. 지금까지는 그걸 기록해야 한다는 생각을 못했는데 이제 와서 정리하려고 보니 내가 했던 게 뭔지, 그걸 통해 뭘 배우고 느꼈는지 잘 기억이 안 나서 막막했다. 그제서야 어떤 형태로든 기록을 해야겠구나 싶었다. 이제 더는 학벌이나 성적이 나를 증명해주지 않는다. 요즘 들어 그걸 정말 많이 느끼고 있고, 앞으로는 세상이 더욱 그런 방향으로 변해갈 것이라 생각한다.저자는 일상의 모든 기록들이 자신을 표현하는 메시지가 된다고 말한다. 이력서나 포트폴리오로 자신을 소개하는 것을 넘어 이제는 소비로도, sns관계망으도 나를 표현하는 시대가 온 것이다. 사람들은 이제 필요에 의해 구매하는 물질소비가 아닌, 자신의 개

In [55]:
# !pip install keybert
# !pip install kiwipiepy

In [56]:
# !pip install sentence_transformers

In [61]:
np.object = object    
np.int = int
np.float = float
np.bool = bool
# import keyBERT np.object 오류 제거 위해 직접 대입 사용

In [62]:
from keybert import KeyBERT

In [63]:
from kiwipiepy import Kiwi

In [64]:
from transformers import BertModel

In [65]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
keywords

[('책이다', 0.4772),
 ('책을', 0.441),
 ('책은', 0.405),
 ('책의', 0.3337),
 ('이야기를', 0.3304),
 ('한국사회를', 0.3293),
 ('말한다', 0.3106),
 ('무능과', 0.2832),
 ('과거부터', 0.2673),
 ('능력을', 0.256)]

In [70]:
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [99]:
# 3개의 단어 묶음인 단어구 추출
n_gram_range = (3, 3)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 364
trigram 다섯개만 출력 : ['sns관계망으도 나를 표현하는' 'sns에서 누구를 팔로우하느냐도' '가장 인상적이었다 요즘' '가족 친구들과 대화하다보면'
 '가진 사람은 그리']


In [100]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [101]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['고집을 날카롭게 지적하는', '날카롭게 지적하는 부분이', '친구들과 대화하다보면 일도', '안주하며 변화를 두려워하고', '가족 친구들과 대화하다보면']


In [89]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [97]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [104]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['의해 구매하는 물질소비가',
 '관심사와 네트워크를 공개하는',
 '고집을 날카롭게 지적하는',
 '친구들과 대화하다보면 일도',
 '가족 친구들과 대화하다보면']

In [103]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20)

['언젠가는 나에게도 변화를',
 '현실에 안주하며 변화를',
 '포트폴리오로 자신을 소개하는',
 '바람이다 중간관리자들의 무능과',
 '친구들과 대화하다보면 일도']

In [102]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['가족 친구들과 대화하다보면',
 '경계하기 위해서도 번쯤',
 '스스로를 경계하기 위해서도',
 '얘기가 나온다 나는',
 '패턴이 이런 식으로']

In [75]:
kiwi = Kiwi()
kiwi.analyze(doc)

[([Token(form='저자', tag='NNG', start=3, len=2),
   Token(form='는', tag='JX', start=5, len=1),
   Token(form='과거', tag='NNG', start=7, len=2),
   Token(form='부터', tag='JX', start=9, len=2),
   Token(form='현재', tag='NNG', start=12, len=2),
   Token(form='까지', tag='JX', start=14, len=2),
   Token(form='변화', tag='NNG', start=17, len=2),
   Token(form='하', tag='XSV', start=19, len=1),
   Token(form='어', tag='EC', start=19, len=1),
   Token(form='오', tag='VX', start=20, len=1),
   Token(form='ᆫ', tag='ETM', start=20, len=1),
   Token(form=',', tag='SP', start=21, len=1),
   Token(form='또', tag='MAG', start=23, len=1),
   Token(form='앞', tag='NNG', start=25, len=1),
   Token(form='으로', tag='JKB', start=26, len=2),
   Token(form='변화', tag='NNG', start=29, len=2),
   Token(form='하', tag='XSV', start=31, len=1),
   Token(form='ᆯ', tag='ETM', start=31, len=1),
   Token(form='한국', tag='NNP', start=33, len=2),
   Token(form='사회', tag='NNG', start=35, len=2),
   Token(form='를', tag='JKO', start=37, 

In [76]:
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [80]:
tokenized_nouns = noun_extractor(doc)
tokenized_nouns

['저자',
 '과거',
 '현재',
 '변화',
 '변화',
 '한국',
 '사회',
 '데이터',
 '이야기',
 '결의',
 '현상',
 '나열',
 '저자',
 '통찰',
 '오랜만',
 '생각',
 '사람',
 '이야기',
 '이해',
 '전달',
 '능력',
 '사람',
 '변화',
 '사회',
 '사회',
 '구성원',
 '개개인',
 '자세',
 '제시',
 '세대',
 '관리자',
 '바람',
 '중간',
 '관리자',
 '무능',
 '고집',
 '지적',
 '부분',
 '사회생활',
 '시작',
 '가족',
 '친구',
 '대화',
 '책임',
 '상사',
 '얘기',
 '어른',
 '스스로',
 '경계',
 '중간중간',
 '문장',
 '당신',
 '메시지',
 '부제',
 '챕터',
 '인상',
 '요즘',
 '기록',
 '중요',
 '체감',
 '개인',
 '기억',
 '어디',
 '활동',
 '생각',
 '지금',
 '그거',
 '기록',
 '생각',
 '정리',
 '그거',
 '기억',
 '형태',
 '기록',
 '학벌',
 '성적',
 '증명',
 '요즘',
 '그거',
 '세상',
 '방향',
 '생각',
 '저자',
 '일상',
 '기록',
 '자신',
 '표현',
 '메시지',
 '이력서',
 '포트폴리오',
 '자신',
 '소개',
 '이제',
 '소비',
 'sns',
 '관계',
 '표현',
 '시대',
 '사람',
 '필요',
 '구매',
 '물질',
 '소비',
 '자신',
 '개성',
 '의미',
 '표현',
 '의미',
 '소비',
 '요즘',
 '소비',
 '패턴',
 '물건',
 '브랜드',
 '추구',
 '가치',
 '지향',
 '경우',
 '마음',
 '소비',
 '해당',
 '브랜드',
 '기억',
 '콘텐츠',
 '소비',
 '마찬가지',
 '콘텐츠',
 '선택',
 '취향',
 '은연중',
 'sns',
 '누구',
 '팔로우',
 '관심사',
 '네트워크',
 '공개',
 '행위',
 '일종',
 '

In [78]:
text = ' '.join(nouns)
text

'저자 과거 현재 변화 변화 한국 사회 데이터 이야기 결의 현상 나열 저자 통찰 오랜만 생각 사람 이야기 이해 전달 능력 사람 변화 사회 사회 구성원 개개인 자세 제시 세대 관리자 바람 중간 관리자 무능 고집 지적 부분 사회생활 시작 가족 친구 대화 책임 상사 얘기 어른 스스로 경계 중간중간 문장 당신 메시지 부제 챕터 인상 요즘 기록 중요 체감 개인 기억 어디 활동 생각 지금 그거 기록 생각 정리 그거 기억 형태 기록 학벌 성적 증명 요즘 그거 세상 방향 생각 저자 일상 기록 자신 표현 메시지 이력서 포트폴리오 자신 소개 이제 소비 sns 관계 표현 시대 사람 필요 구매 물질 소비 자신 개성 의미 표현 의미 소비 요즘 소비 패턴 물건 브랜드 추구 가치 지향 경우 마음 소비 해당 브랜드 기억 콘텐츠 소비 마찬가지 콘텐츠 선택 취향 은연중 sns 누구 팔로우 관심사 네트워크 공개 행위 일종 저자 결국 수렴 라이프 스타일 메시지 한편 기록 수단 비밀 사회 지금 현실 안주 변화 거부 변화 해당 생각 저자 서문 이야기 변화 변화 준비 장단점 세상 장점 조금'

In [109]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('저자', 0.5892),
 ('얘기', 0.5879),
 ('요즘', 0.5826),
 ('소개', 0.5788),
 ('이야기', 0.5707),
 ('수렴', 0.5695),
 ('세상', 0.5643),
 ('지금', 0.5524),
 ('팔로우', 0.5369),
 ('문장', 0.529),
 ('어디', 0.5223),
 ('필요', 0.5217),
 ('제시', 0.5212),
 ('안주', 0.5194),
 ('사람', 0.5178),
 ('결의', 0.5172),
 ('친구', 0.5117),
 ('이제', 0.5109),
 ('행위', 0.5068),
 ('그거', 0.5055)]

In [138]:
keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,2),use_maxsum = True,top_n = 10, highlight=False)
keywords

[('표현하는 메시지가', 0.47),
 ('현재까지 변화해온', 0.4718),
 ('책의 부제이기도', 0.4733),
 ('일상의 모든', 0.4757),
 ('단순히 현상만', 0.4822),
 ('전달하는 능력을', 0.4886),
 ('나의 관심사와', 0.4914),
 ('책은 변화하는', 0.5083),
 ('활동들과 생각을', 0.5415),
 ('중간관리자들의 무능과', 0.6187)]

In [130]:
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(2,4),use_maxsum = True,top_n = 20)
keywords

[('언젠가는 나에게도 변화를 맞이할', 0.5852),
 ('현상만 나열하는 아니라 저자의', 0.5858),
 ('구매하는 물질소비가 아닌 자신의', 0.5862),
 ('활동들과 생각을 보여줘야할 특히', 0.5867),
 ('하는 바람이다 중간관리자들의', 0.5868),
 ('언젠가는 나에게도 변화를', 0.5884),
 ('않았다 해도 변화는 분명히', 0.5911),
 ('바람이다 중간관리자들의', 0.5914),
 ('읽어봤으면 하는 바람이다 중간관리자들의', 0.5915),
 ('가족 친구들과 대화하다보면 일도', 0.5921),
 ('해도 변화는 분명히', 0.5933),
 ('있다 언젠가는 나에게도 변화를', 0.5942),
 ('바람이다 중간관리자들의 무능과 고집을', 0.6052),
 ('생각한다 저자는 일상의', 0.6072),
 ('해도 변화는 분명히 일어나고', 0.6094),
 ('것이라 생각한다 저자는 일상의', 0.6146),
 ('중간관리자들의 무능과', 0.6187),
 ('어딘가에 내가 활동들과 생각을', 0.6226),
 ('바람이다 중간관리자들의 무능과', 0.623),
 ('하는 바람이다 중간관리자들의 무능과', 0.6288)]

In [133]:
keywords_mmr = kw_model.extract_keywords(doc,keyphrase_ngram_range=(2,3),use_mmr = True,top_n = 20,diversity = 0.5)
keywords_mmr

[('바람이다 중간관리자들의 무능과', 0.623),
 ('생각한다 저자는 일상의', 0.6072),
 ('해도 변화는 분명히', 0.5933),
 ('언젠가는 나에게도 변화를', 0.5884),
 ('하는 바람이다 중간관리자들의', 0.5868),
 ('중간관리자들의 무능과 고집을', 0.5705),
 ('활동들과 생각을', 0.5415),
 ('좋았다 책은 변화하는', 0.5205),
 ('변화해온 앞으로 변화할', 0.5159),
 ('대화하다보면 일도 못하면서', 0.5129),
 ('팔로우하느냐도 나의 관심사와', 0.5055),
 ('나열하는 아니라 저자의', 0.5012),
 ('개성을 드러내고 의미를', 0.4814),
 ('포트폴리오로 자신을 소개하는', 0.4448),
 ('나의 관심사와 네트워크를', 0.4387),
 ('들어 기록의 중요성을', 0.4192),
 ('비슷한 결의 책을', 0.4021),
 ('있을테니 조금은 무서워해도', 0.3831),
 ('절실히 느낀다', 0.3396),
 ('같기도 하다', 0.2738)]

In [81]:
#명사로 trigram 만들기

In [83]:
tokenized_nouns

['저자',
 '과거',
 '현재',
 '변화',
 '변화',
 '한국',
 '사회',
 '데이터',
 '이야기',
 '결의',
 '현상',
 '나열',
 '저자',
 '통찰',
 '오랜만',
 '생각',
 '사람',
 '이야기',
 '이해',
 '전달',
 '능력',
 '사람',
 '변화',
 '사회',
 '사회',
 '구성원',
 '개개인',
 '자세',
 '제시',
 '세대',
 '관리자',
 '바람',
 '중간',
 '관리자',
 '무능',
 '고집',
 '지적',
 '부분',
 '사회생활',
 '시작',
 '가족',
 '친구',
 '대화',
 '책임',
 '상사',
 '얘기',
 '어른',
 '스스로',
 '경계',
 '중간중간',
 '문장',
 '당신',
 '메시지',
 '부제',
 '챕터',
 '인상',
 '요즘',
 '기록',
 '중요',
 '체감',
 '개인',
 '기억',
 '어디',
 '활동',
 '생각',
 '지금',
 '그거',
 '기록',
 '생각',
 '정리',
 '그거',
 '기억',
 '형태',
 '기록',
 '학벌',
 '성적',
 '증명',
 '요즘',
 '그거',
 '세상',
 '방향',
 '생각',
 '저자',
 '일상',
 '기록',
 '자신',
 '표현',
 '메시지',
 '이력서',
 '포트폴리오',
 '자신',
 '소개',
 '이제',
 '소비',
 'sns',
 '관계',
 '표현',
 '시대',
 '사람',
 '필요',
 '구매',
 '물질',
 '소비',
 '자신',
 '개성',
 '의미',
 '표현',
 '의미',
 '소비',
 '요즘',
 '소비',
 '패턴',
 '물건',
 '브랜드',
 '추구',
 '가치',
 '지향',
 '경우',
 '마음',
 '소비',
 '해당',
 '브랜드',
 '기억',
 '콘텐츠',
 '소비',
 '마찬가지',
 '콘텐츠',
 '선택',
 '취향',
 '은연중',
 'sns',
 '누구',
 '팔로우',
 '관심사',
 '네트워크',
 '공개',
 '행위',
 '일종',
 '

In [82]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
#transformer and sentencepiece

In [110]:
!pip install transformers
!pip install sentencepiece

In [113]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
from collections import Counter

# 한국어 BERT 모델과 토크나이저 로드 (여기서는 'bert-base-multilingual-cased' 사용)
model_name = "bert-base-multilingual-cased"
# model_name = 'skt/kobert-base-v1'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# 입력 문서
# document = "한국어 문서에서 키워드 추출 예제입니다."
document = doc

# 문장을 토큰화하고 [MASK] 토큰 삽입
tokens = tokenizer.tokenize(document)
masked_index = tokens.index("[MASK]") if "[MASK]" in tokens else -1

if masked_index == -1:
    # 문장에 [MASK] 토큰이 없는 경우, 끝에 추가
    tokens += ["[MASK]"]
else:
    # [MASK] 토큰이 이미 있는 경우, 제거하고 끝에 추가
    tokens[masked_index] = "[MASK]"

# BERT 모델 입력 형식으로 변환
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_tensor = torch.tensor(input_ids).unsqueeze(0)  # 배치 차원 추가

# BERT 모델로부터 확률 분포 얻기
with torch.no_grad():
    outputs = model(input_tensor)
    predictions = outputs.logits

# [MASK] 위치의 예측 확률 분포에서 상위 N 단어 추출
N = 5
top_k_tokens = torch.topk(predictions[0, masked_index], N).indices.tolist()

# 상위 N 단어를 토큰으로 변환하여 출력
keywords = [tokenizer.decode([token]) for token in top_k_tokens]
print(keywords)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: The expanded size of the tensor (992) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 992].  Tensor sizes: [1, 512]

In [116]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
from collections import Counter

# 한국어 BERT 모델과 토크나이저 로드 (여기서는 'bert-base-multilingual-cased' 사용)
# model_name = "bert-base-multilingual-cased"
model_name = 'skt/kobert-base-v1'
# tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertForMaskedLM.from_pretrained(model_name)
model = BertModel.from_pretrained('skt/kobert-base-v1')

# 입력 문서
# document = "한국어 문서에서 키워드 추출 예제입니다."
document = doc
# 시퀀스 길이와 배치 크기 조절
max_seq_length = 128  # 최대 시퀀스 길이
batch_size = 32  # 배치 크기

# 문장을 토큰화하고 [MASK] 토큰 삽입
tokens = kiwi.analyze(doc) #kiwi로 토크나이즈
#tokens = tokenizer.tokenize(document)[:max_seq_length]
masked_index = tokens.index("[MASK]") if "[MASK]" in tokens else -1

if masked_index == -1:
    # 문장에 [MASK] 토큰이 없는 경우, 끝에 추가
    tokens += ["[MASK]"]
else:
    # [MASK] 토큰이 이미 있는 경우, 제거하고 끝에 추가
    tokens[masked_index] = "[MASK]"

# BERT 모델 입력 형식으로 변환
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_tensor = torch.tensor(input_ids).unsqueeze(0)  # 배치 차원 추가

# 배치 크기 조절 (예: 32개의 문장을 동시에 처리하려면 여기서 배치 크기를 32로 설정)
input_tensor = input_tensor.repeat(batch_size, 1)

# BERT 모델로부터 확률 분포 얻기
with torch.no_grad():
    outputs = model(input_tensor)
    predictions = outputs.logits

# [MASK] 위치의 예측 확률 분포에서 상위 N 단어 추출
N = 5
top_k_tokens = torch.topk(predictions[:, masked_index], N).indices.tolist()

# 상위 N 단어를 토큰으로 변환하여 출력
keywords = [tokenizer.decode([token]) for token in top_k_tokens[0]]
print(keywords)


TypeError: unhashable type: 'list'

In [ ]:
# 전체 추출 함수
def kiwi_token_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [124]:
from kiwipiepy import Kiwi
import torch
from transformers import BertTokenizer, BertForMaskedLM
from collections import Counter


kiwi = Kiwi()
#kiwi.prepare()

# sentence = "한국어 문장을 형태소 분석하기"
document = doc
# 시퀀스 길이와 배치 크기 조절
max_seq_length = 128  # 최대 시퀀스 길이
batch_size = 32  # 배치 크기

result = kiwi.analyze(document)

#token 뽑기
tokens = [token for token in result[0][0]]

model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

#input_ids = tokenizer.convert_tokens_to_ids(tokens)
#input_tensor = torch.tensor(input_ids).unsqueeze(0)  # 배치 차원 추가

# BERT 모델 입력 형식으로 변환
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_tensor = torch.tensor(input_ids).unsqueeze(0)  # 배치 차원 추가

# 배치 크기 조절 (예: 32개의 문장을 동시에 처리하려면 여기서 배치 크기를 32로 설정)
input_tensor = input_tensor.repeat(batch_size, 1)

with torch.no_grad():
    outputs = model(input_tensor)
    predictions = outputs.logits

# 키워드 추출 또는 다른 작업 수행


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: The expanded size of the tensor (920) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [32, 920].  Tensor sizes: [1, 512]

In [125]:
from kiwipiepy import Kiwi
import torch
from transformers import BertTokenizer, BertForMaskedLM
from collections import Counter

# 한국어 BERT 모델과 토크나이저 로드 (여기서는 'bert-base-multilingual-cased' 사용)
# model_name = "bert-base-multilingual-cased"
model_name = 'skt/kobert-base-v1'
# tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertForMaskedLM.from_pretrained(model_name)
model = BertModel.from_pretrained('skt/kobert-base-v1')

kiwi = Kiwi()
#kiwi.prepare()

# 입력 문서
# document = "한국어 문서에서 키워드 추출 예제입니다."
document = doc
# 시퀀스 길이와 배치 크기 조절
max_seq_length = 128  # 최대 시퀀스 길이
batch_size = 32  # 배치 크기

result = kiwi.analyze(document)

#token 뽑기
tokens = [token for token in result[0][0]]


# 문장을 토큰화하고 [MASK] 토큰 삽입
tokens = kiwi.analyze(doc) #kiwi로 토크나이즈
#tokens = tokenizer.tokenize(document)[:max_seq_length]
masked_index = tokens.index("[MASK]") if "[MASK]" in tokens else -1

if masked_index == -1:
    # 문장에 [MASK] 토큰이 없는 경우, 끝에 추가
    tokens += ["[MASK]"]
else:
    # [MASK] 토큰이 이미 있는 경우, 제거하고 끝에 추가
    tokens[masked_index] = "[MASK]"

# BERT 모델 입력 형식으로 변환
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_tensor = torch.tensor(input_ids).unsqueeze(0)  # 배치 차원 추가

# 배치 크기 조절 (예: 32개의 문장을 동시에 처리하려면 여기서 배치 크기를 32로 설정)
input_tensor = input_tensor.repeat(batch_size, 1)

# BERT 모델로부터 확률 분포 얻기
with torch.no_grad():
    outputs = model(input_tensor)
    predictions = outputs.logits

# [MASK] 위치의 예측 확률 분포에서 상위 N 단어 추출
N = 5
top_k_tokens = torch.topk(predictions[:, masked_index], N).indices.tolist()

# 상위 N 단어를 토큰으로 변환하여 출력
keywords = [tokenizer.decode([token]) for token in top_k_tokens[0]]
print(keywords)

TypeError: unhashable type: 'list'

In [129]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
from kiwipiepy import Kiwi

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# 한국어 BERT 모델과 토크나이저 로드 (여기서는 'bert-base-multilingual-cased' 사용)
# model_name = "bert-base-multilingual-cased"
model_name = 'skt/kobert-base-v1'
#tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
keywords

# 입력 문서
document = "한국어 문서에서 키워드 추출 예제입니다."

# Kiwi로 문장을 형태소 분석하고 토큰화
result = kiwi.analyze(document)
#tokens = [morph.lex for word in result[0][0]]
tokens = [token for token in result[0][0]]

# BERT 모델 입력 형식으로 변환
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_tensor = torch.tensor(input_ids).unsqueeze(0)  # 배치 차원 추가

# 텐서 크기 조절 (예: 시퀀스 길이를 128로, 배치 크기를 1로 설정)
max_seq_length = 128
batch_size = 1
input_tensor = input_tensor[:, :max_seq_length]  # 시퀀스 길이 제한
input_tensor = input_tensor.repeat(batch_size, 1)  # 배치 크기 조절

# BERT 모델로부터 확률 분포 얻기
with torch.no_grad():
    outputs = model(input_tensor)
    predictions = outputs.logits

# [MASK] 위치의 예측 확률 분포에서 상위 N 단어 추출
masked_index = tokens.index("[MASK]") if "[MASK]" in tokens else -1
N = 5
top_k_tokens = torch.topk(predictions[:, masked_index], N).indices.tolist()

# 상위 N 단어를 토큰으로 변환하여 출력
keywords = [tokenizer.decode([token]) for token in top_k_tokens[0]]
print(keywords)



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'BertTokenizer'.


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType